# Tâche 2 : Question-réponse avec modèle pré‑entraîné GPT‑2, sans adaptation

**Objectifs**

Observer les limites d’un modèle de langage **pré‑entraîné** (**GPT-2 Medium (355M)**) lorsqu’on lui pose des questions sur un sujet **absent** de ses données d’origine (ici, l’univers de *Sherlock Holmes*).
Dans ce *notebook*, vous faites **uniquement de l’inférence** avec un modèle qui a déjà été pré-entraîné. Cette tâche consiste à construire un *prompt* minimal, générer des réponses avec le modèle sans modification, puis évaluer la pertinence des résultats. Aucun entraînement de modèle n'est effectué pour cette tâche.

**Objectifs d’apprentissage**
1. Charger un modèle pré‑entraîné et son tokenizer de Hugging Face.
2. Générer du texte et **isoler la réponse** du modèle.
3. Comprendre et expliquer les **limites du pré‑entraînement** hors‑domaine.

Les **questions** pour évaluer le modèle vous sont fournies. Vous devez comprendre le format des questions chargées en mémoire.

> Il est recommandé de faire ce travail pratique en utilisant une carte graphique GPU compatible avec HuggingFace/Pytorch.
> Si votre machine n’en possède pas, vous pouvez utiliser **Google Colab** pour exécuter le *notebook* dans le cloud.

Si nécessaire, installer les *packages* suivant. Si vous exécutez sur Code Colab, ces *packages* devraient déjà être installés.

In [ ]:
#!pip install datasets
#!pip install accelerate
#!pip install transformers[torch]
#!pip3 install torch torchvision

In [ ]:
from transformers import pipeline
import os
import json

Ici, j'ajoute du code afin de pouvoir sauvegarder mes résultats en dehors de colab. À ignorer si le notebook est exécuté en local. Sinon il peux être désirable d'ajuster la variable source pour pointer le dossier google drive à utiliser.

In [ ]:
try:
  from google.colab import drive
  IN_COLAB = True
except:
  IN_COLAB = False

#Mount a google drive folder to save things
if IN_COLAB:
  drive.mount('/content/drive')
  folders_to_mount = ["nlp_tp2_models", "results"]
  for folder in folders_to_mount:
    source = f'/content/drive/MyDrive/uni/nlp/{folder}'
    shortcut = f'/content/{folder}'
    print(f"Mounting {source} to {shortcut}")
    os.symlink(source, shortcut)

#clone repo to access data
repo_url = "https://github.com/XavyShmore/tp2_nlp.git"
if IN_COLAB:
  !git clone {repo_url}
  !cp -r ./tp2_nlp/data .
  pass

Mounted at /content/drive
Mounting /content/drive/MyDrive/uni/nlp/nlp_tp2_models to /content/nlp_tp2_models
Mounting /content/drive/MyDrive/uni/nlp/results to /content/results
Cloning into 'tp2_nlp'...
remote: Enumerating objects: 22, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 22 (delta 8), reused 11 (delta 3), pack-reused 0 (from 0)
Receiving objects: 100% (22/22), 1.47 MiB | 26.87 MiB/s, done.
Resolving deltas: 100% (8/8), done.


In [ ]:
batch_size = 5 # il est possible d'ajuster la taille de batch. Les valeurs actuelles utilisent environ 10 Gb
max_length = 256 # on réduit le contexte pour sauver du temps, nos exemples ne nécesside pas une plus grande fenêtre de mots
model_name = "gpt2-medium"

## 1. Chargement du modèle Hugging Face et du tokenizer (à compléter)

Complétez le corps de la fonction `load_model(model_path)` afin qu’elle :

- charge le **tokenizer** et le **modèle** Hugging Face à partir du chemin `model_path`.
- **retourne** le tokenizer comme **première valeur de retour** et le modèle comme **seconde valeur de retour**.

On ajoute également des fonctions pour monter les questions en mémoire et pour sauvegarder les réponses dans un fichier.

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

def load_model(model_path):
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    model = AutoModelForCausalLM.from_pretrained(model_path)

    return tokenizer, model

In [ ]:
def load_entries(path):
    with open(path, "r", encoding="utf-8") as file:
        data = json.load(file)
    if not isinstance(data, list):
        raise ValueError(f"Question file must contain a list of objects. Got: {type(data)}")
    return data

def save_answers(questions_answers, output_dir, out_file_name, display=True):
    os.makedirs(output_dir, exist_ok=True)
    with open(os.path.join(output_dir, out_file_name), "w", encoding="utf-8") as out:
        for index, question, answer, expected_answer in questions_answers:
            out.write(f"Q: {question}\nA: {answer}\nExpected:{expected_answer}\n{'-' * 60}\n")
            if display:
                print(f"Q{index}: {question}\nA: {answer}\nExpected:{expected_answer}\n{'-' * 60}")


## 2. Fonctions de test question-réponse  (à compléter)

La fonction **test_on_questions** est utilisée pour parcourir **toutes les entrées** du fichier de questions afin de produire des réponses générées par le modèle.

La génération d'une réponse à une question implique les étapes suivantes (fonction **process_entry** à compléter) :
* Construire un prompt à l’aide de la fonction **build_prompt** (rendu disponible).
* Utiliser le modèle (via un pipeline de génération passé en argument) pour générer une réponse à une question.
* Retourner la réponse générée par le modèle.  

Points importants à souligner:
* La fonction ***process_entry*** doit retourner uniquement la réponse générée par le modèle (sans la question - le prompt).
* Il est de votre responsabililté de choisir **les paramètres** du générateur (max_new_tokens, do_sample, temperature, top_k ou top_p). Décrivez ceux que vous avez retenus.

> Afin de simplifier le travail, nous avons choisi de ne pas utiliser de *batchs* dans la fonction qui teste les questions.
> Vous n'avez pas à prendre en compte le *warning* qui suggère d'utiliser des *datasets*.

In [ ]:
def build_prompt(entry):
    question = entry.get("question", "")
    prompt = f"Please answer the following question about the Sherlock Holmes Universe with a short answer.\nQuestion: {question}\nAnswer: "
    return prompt

In [ ]:
entries = load_entries("data/questions_sherlock.json")
print(build_prompt(entries[0]))

Please answer the following question about the Sherlock Holmes Universe with a short answer.
Question: Where do Sherlock Holmes and Dr. Watson live?
Answer: 


Description des paramètres de génération:

`max_new_tokens = 10`: Car il n'y a pas de réponse qui nécéssite plus long pour répondre. De plus, raccourcir permet de réduit le temps de calcul.
`do_sample = False`: permet des réponse plus factuelle. Empèche le modèle de s'égarer.

Puisque do_sample = False, la température, top_k ne sont pas nécessaire.

`num_return_sequences=1` On ne veux qu'une seule réponse.

In [ ]:
def process_entry(entry, prompt_builder, generator):
    prompt = prompt_builder(entry)
    generation_output = generator(
        prompt,
        max_new_tokens=10,
        do_sample=False,
        temperature=0.8,
        top_k=50,
        num_return_sequences=1
    )
    # The generated_text includes the prompt, so we need to extract only the answer
    generated_text = generation_output[0]['generated_text']
    answer = generated_text[len(prompt):].strip()
    return answer

In [ ]:
def test_on_questions(prompt_builder, model_path, question_file, out_file_name, output_dir="results"):
    entries = load_entries(question_file)
    tokenizer, model = load_model(model_path)
    generator = pipeline("text-generation", model=model, tokenizer=tokenizer)
    results = []
    for i, entry in enumerate(entries):
        answer = process_entry(entry, prompt_builder, generator)
        question = entry.get("question", "")
        expected_answer = entry.get("answer", "")
        results.append((i, question, answer, expected_answer))
    save_answers(results, output_dir, out_file_name, display=True)
    return results

## 3. Génération avec GPT-2 de réponses aux questions sur Sherlock Holmes

Exécutez la cellule suivante pour générer les réponses aux questions.
Le temps d’exécution devrait se situer entre **5 et 10 minutes** si vous utilisez **Google Colab** avec un GPU.

Note : N'oubliez pas d'ajouter le fichier de réponses générées par le modèle (voir *out_file_name*) dans votre remise du travail. Ainsi le fichier ZIP que vous déposerez sur le site du cours devra contenir tous les *notebooks* et tous les fichiers de réponses.

In [ ]:
questions = "data/questions_sherlock.json"
out_file_name = "gpt2_answers.txt"

results = test_on_questions(prompt_builder=build_prompt, model_path=model_name, question_file=questions, out_file_name=out_file_name)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/718 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.52G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Device set to use cuda:0
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
Setting `pad

Q0: Where do Sherlock Holmes and Dr. Watson live?
A: Sherlock Holmes lives in London, England.
Expected:221B Baker Street, London.
------------------------------------------------------------
Q1: Who is Sherlock Holmes' loyal friend and chronicler?
A: The answer is Sherlock Holmes' loyal friend and
Expected:Dr. John H. Watson.
------------------------------------------------------------
Q2: Who is considered 'The Woman' by Sherlock Holmes?
A: The Woman is the woman who is the most
Expected:Irene Adler.
------------------------------------------------------------
Q3: Which story features the Red-Headed League?
A: The Red-Headed League.
Question
Expected:The Adventure of the Red-Headed League.
------------------------------------------------------------
Q4: What is the primary occupation of Sherlock Holmes?
A: Sherlock Holmes is a detective.
Question
Expected:Consulting detective.
------------------------------------------------------------
Q5: Who is Sherlock Holmes' arch-nemesis?
A: Th

## 4. Analyse des résultats

### 4.1 Évaluation quantitative (à compléter)

In [ ]:
import string
import re
from collections import Counter

def remove_articles(text):
    return re.sub(r'\b(a|an|the)\b', ' ', text)

def white_space_fix(text):
    return ' '.join(text.split())

def remove_punc(text):
    exclude = set(string.punctuation)
    return ''.join(ch for ch in text if ch not in exclude)

def lower(text):
    return text.lower()

def normalize_answer(s):
    """Mettre en minuscule et retirer la ponctuation, des déterminants and les espaces."""
    return white_space_fix(remove_articles(remove_punc(lower(s))))

In [ ]:
def evaluate_f1(ground_truth, prediction):
    """Normalise les 2 textes, trouve ce qu'il y a en commun et estime précision, rappel et F1."""
    prediction_tokens = normalize_answer(prediction).split()
    ground_truth_tokens = normalize_answer(ground_truth).split()
    common = Counter(prediction_tokens) & Counter(ground_truth_tokens)
    num_same = sum(common.values())
    if num_same == 0:
        return 0.0, 0.0, 0.0
    precision = 1.0 * num_same / len(prediction_tokens)
    recall = 1.0 * num_same / len(ground_truth_tokens)
    f1 = (2 * precision * recall) / (precision + recall)
    return precision, recall, f1

In [ ]:
def evaluation_generation(results):
    eval = {"precision":0, "recall":0, "f1":0}
    for i, question, answer, expected_answer in results:
        precision, recall, f1 = evaluate_f1(expected_answer, answer)
        eval["precision"] += precision
        eval["recall"] += recall
        eval["f1"] += f1

    eval["precision"] /= len(results)
    eval["recall"] /= len(results)
    eval["f1"] /= len(results)

    return eval

In [ ]:
eval = evaluation_generation(results)
print(eval)

{'precision': 0.055523809523809524, 'recall': 0.145, 'f1': 0.07263636363636362}


**Question:** Que pensez-vous de cette évaluation ?

Ces résultats ne sembles pas terribles.

En effet, le fait qu'en moyenne moins de 6% de mots de la prédiction soit présent dans la réponse (précision) n'est pas super.

Que le rapel soit de 0.145 en moyenne me semble meilleur que la précision. 14,5% des mots des prédictions se trouve effectivement dans la réponse. Le fait que cette métrique soit meilleur semble indiquer que les réponses du modèle sont trop longue et contienne de nombreux mots superflux même si parfois la réponse est bonne.

Bref, un score f1 de 0.0726 ne semble pas très bon pour cette tâche.

### 4. Analyse qualitative (à faire)

Rédigez **5 à 8 phrases** expliquant ce que vous observez et pourquoi, selon vous, le modèle fournit ce type de réponses.

> Cette étape prépare le terrain pour les tâches 3 et 4.
> Il est normal que les réponses ne soient pas très bonnes à ce stade. On vous demande d’expliquer **pourquoi**.